<a href="https://colab.research.google.com/github/fezjo/ml-project/blob/master/ml_project_ciz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/fezjo/ml-project/raw/master/chat_history.zip
!unzip -Pputpassword chat_history.zip
!pip install unidecode

--2023-01-30 22:49:22--  https://github.com/fezjo/ml-project/raw/master/chat_history.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fezjo/ml-project/master/chat_history.zip [following]
--2023-01-30 22:49:23--  https://raw.githubusercontent.com/fezjo/ml-project/master/chat_history.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2804699 (2.7M) [application/zip]
Saving to: ‘chat_history.zip.10’

chat_history.zip.10 100%[===================>]   2.67M  --.-KB/s    in 0.009s  

2023-01-30 22:49:23 (291 MB/s) - ‘chat_history.zip.10’ saved [2804699/2804699]

Archive:  chat_history.zip
   sk

In [2]:
from matplotlib import pyplot as plt

def plot_history(history):
  dpi = 100
  plt.figure(dpi=dpi)
  plt.plot(history.history['loss'], label='training loss')
  plt.plot(history.history.get('val_loss', []), label='validation loss')
  plt.legend(loc='best')

  plt.figure(dpi=dpi)
  plt.plot(history.history['accuracy'], label='train accuracy')
  plt.plot(history.history.get('val_accuracy', []), label='validation accuracy')
  plt.legend(loc='best')
  plt.show()

In [3]:
import json

with open("chat_history.json") as chf:
  data = json.load(chf)

In [4]:
from unidecode import unidecode

author_blacklist = {"PollBot", "Dartboi", "Telelog"}
authors = dict()
messages = []
for message in data["messages"]:
  if message["type"] != "message":
    continue
  text_entities, author = message["text_entities"], message["from"]
  if not text_entities or author in author_blacklist:
    continue
  text_chunks = []
  for entity in text_entities:
    text_chunks.append(entity["text"])
  text = "".join(text_chunks)
  text = unidecode(text).lower()
  authors.setdefault(author, len(authors))
  messages.append([text, authors[author]])

print(authors)
print(messages[:100])

{'Jozef': 0, 'Bohdanator': 1, 'Daniel Oravec': 2, 'Samuel Čavoj': 3, 'Dávid Mišiak': 4, 'Pavol Kebis': 5, 'Jaroslav Paška': 6, 'Ján Priner': 7}
[['make me admin pls', 0], ['sure :d :d', 1], ['aj mna!!', 2], ['aj mna!!!', 3], ['', 3], ['wtf', 1], ['ono to islo dokelu tak rychlo...', 1], ['neviem, co si cakal, ze sa stane, ked z jozefa spravis admina', 3], ['nespravil som :d :d', 1], ['jaj', 3], ['zabudol som nastavit aby sa to nedialo', 1], [':(', 2], ['genialne', 3], ['kde su vsetky prvacky?', 0], ['co tu robi paska?', 3], ['flexi', 0], ['ani nie', 3], ['tu neexistuje demokracia', 1], ['preco ste dnes neboli v skole?', 2], ['just kidding', 1], ['podme sa bavit o kamenoch', 1], ['lebo som bol na orave', 1], ['rebels', 4], ['pridte na vba231', 0], ['?', 4], ['kedy sa zacina skola?', 0], ['vyzera to tak ze v pondelok', 4], ['cas?', 0], ['asi podla rozvrhu', 4], ['alebo to mecheche otvorenie akademickeho roka', 4], ['neviem ktore z toho', 4], ['vie niekto?', 4], ['podla mna normalne podla 

In [5]:
author_activity = [0] * len(authors)
for message in messages:
  author_activity[message[1]] += 1

for author, activity in zip(authors.items(), author_activity):
  print("{}: {} [{:.2%}]".format(author, activity, activity / len(messages)))

('Jozef', 0): 17808 [27.98%]
('Bohdanator', 1): 6326 [9.94%]
('Daniel Oravec', 2): 6078 [9.55%]
('Samuel Čavoj', 3): 17254 [27.11%]
('Dávid Mišiak', 4): 15336 [24.10%]
('Pavol Kebis', 5): 489 [0.77%]
('Jaroslav Paška', 6): 21 [0.03%]
('Ján Priner', 7): 328 [0.52%]


In [6]:
import numpy as np
message_lengths = np.array(list(map(lambda m: len(m[0]), messages)))
ml_avg, ml_std = np.mean(message_lengths), np.std(message_lengths)
print(f"message length / avg: {ml_avg:.2f} / std: {ml_std:.2f}")

message length / avg: 34.47 / std: 60.23


In [7]:
shuffled_messages = np.random.permutation(messages)
X, Y = map(lambda k: np.array([m[k] for m in shuffled_messages]), (0, 1))
train_test_ratio = 0.8
num_train = int(train_test_ratio * len(X))
X_train, X_test = np.split(X, [num_train])
Y_train, Y_test = np.split(Y, [num_train])
output_shape = len(authors)

print(X[:100])
print(Y[:100])
print(X.shape, X_train.shape, X_test.shape)
print(Y.shape, Y_train.shape, Y_test.shape)

['lol' 'nefunguje mi to'
 'pondelok 8 piatok 8 a este 4 niekam musim napchat'
 'mozete sa mi vysmiat, budem mat 0 za du z foje btw'
 'a potom bohdan povedal tau gama :d'
 'checkpointing\n...\n1. zapis vsetky log-zaznamy na disk'
 ' 1. t vx f(g(x)) = x                               s+  \n 2.*t 3x 3y (g(x) = g(y) /\\ !x = y)                s+  \n 3.*t 3y (g(a) = g(y) /\\ !a = y)                   d2 {x -> a}\n 4.*t (g(a) = g(b) /\\ !a = b)                      d3 {y -> b}\n 5. t g(a) = g(b)                                  a4 (prva cast alfy)\n 6.*t !a = b                                       a4 (druha cast alfy)\n 7. f a = b                                        a6  \n 8. t f(g(a)) = a                                  j1 {x -> a}\n 9. t f(g(b)) = b                                  j1 {x -> b}\n10. t f(g(a)) = b                                  take to pravidlo pre tabla s rovnostou, ze ked a = b, tak mozem niekde za a substituovat b podla 5. do 9. {g(b) -> g(a)}\n11. t a = b          

In [46]:
import tensorflow as tf
from tensorflow import keras


# Use the TextVectorization layer to tokenize the input string based on bigrams
bigram_sequence_dim = int(ml_avg + ml_std)
preprocessing = keras.layers.TextVectorization(
    standardize=None,
    split=tf.strings.bytes_split,
    ngrams=2,
    output_sequence_length=bigram_sequence_dim)
preprocessing.adapt(X_train)

# Input layer for string data
inputs = keras.layers.Input(shape=(1,), dtype=tf.string)

# Preprocessing layer
layer = preprocessing(inputs)

# Embedding layer
max_vocab_size = len(preprocessing.get_vocabulary()) + 1
print(f"Vocabulary size: {max_vocab_size}")
embedding_dim = 200
layer = keras.layers.Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(layer) # TODO regularizer

# Add a dropout layer to prevent overfitting
layer = keras.layers.Dropout(rate=0.3)(layer)

# Add dimension for channel needed by convolution
layer = keras.layers.Reshape(target_shape=(*layer.shape[-2:], 1))(layer)

# # Create 3 parallel branches for convolution with varying window widths
n_conv_filters = 20 # TODO
parent_layer = layer
child_layers = []
for ks in (3, 4, 5):
    layer = parent_layer

    layer = keras.layers.Conv2D(
        filters=n_conv_filters,
        kernel_size=(ks, embedding_dim),
        activation="selu")(layer)
    
    # Max pooling layer
    layer = keras.layers.GlobalMaxPooling2D()(layer)
    child_layers.append(layer)

# Concatenate the outputs of the parallel Conv1D layers
layer = keras.layers.Concatenate()(child_layers)

# # Add a dropout layer to prevent overfitting
# layer = keras.layers.Dropout(rate=0.3)(layer)

# # Add a dense layer
# layer = keras.layers.Dense(units=300, activation="relu")(layer)

# Output layer
outputs = keras.layers.Dense(units=output_shape, activation="softmax")(layer)

# Create the model
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])
model.summary()

Vocabulary size: 3550
Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization_16 (TextVec  (None, 50)          0           ['input_17[0][0]']               
 torization)                                                                                      
                                                                                                  
 embedding_16 (Embedding)       (None, 50, 200)      710000      ['text_vectorization_16[0][0]']  
                                                                                                  
 dropout_16 (Dropout)           (None, 50, 200)      0           ['em

In [12]:
print(preprocessing.get_vocabulary()[:1000])

['', '[UNK]', ' ', 'a', 'e', 'o', 't', 'i', 'n', 's', 'm', 'l', 'r', 'd', 'k', 'e  ', 'p', 'u', 'v', 'c', 'z', 'o  ', 'a  ', '  t', 'j', '  s', 'y', 'b', 'h', '  n', 't o', 'n e', '  p', 'm  ', '  a', 'i  ', 'a k', 'a l', '  m', 'n a', 't a', 'i e', '  z', 's t', 't e', 'p r', '  v', 's i', 'y  ', ',', 'l e', 'o m', ',  ', 'n i', 'p o', 'r a', 'j e', 'm a', 'r e', '?', 'k o', 'a t', 'e d', 't  ', '  d', 'f', '  j', 'o v', '  c', 'u  ', 'z e', 'e n', 'l  ', '  b', 'a m', 'a s', 'e s', 's a', 'r o', 'c h', 'i n', 'e m', 'd a', 'a n', 'd e', 'c i', 's  ', 'k  ', '  k', 'l a', 'v e', 'v i', 's o', 'e r', 't i', 'v a', 'n o', 'i s', 'd o', 'c o', 'r i', 'a v', '  o', 'o l', 'i t', 'o r', 'g', 'o d', 'h o', 'b o', '.', 'e l', 'l o', '  r', 'z  ', 'j a', 'l i', 'a c', 'o z', 'm e', 'n  ', 'k e', 'e c', 'z a', 'm i', 'j  ', 'a j', 'e t', 'o s', 'k a', 'e j', 'd  ', 's e', '0', '  i', 'e b', 'c e', 'm o', 'i a', '  u', 't r', 'o b', 'i l', 'a d', '  h', '1', 'o n', 'a z', 'i m', 's k', 'o t', '

In [27]:
sample_text = "fakin ak si nic nedas, tak PON UTO PIA mas???!? cely den na pracu a este streda poobede :D 👍 @fezjo"

inp = model.input                                           # input placeholder
model_outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = lambda outputs: keras.backend.function(inp, outputs)   # evaluation function

# Testing
for output in model_outputs:
  print(functor([output])(np.array([sample_text])))

[array([[b'fakin ak si nic nedas, tak PON UTO PIA mas???!? cely den na pracu a este streda poobede :D \xf0\x9f\x91\x8d @fezjo']],
      dtype=object)]
[array([[  65,    3,   14,    7,    8,    2,    3,   14,    2,    9,    7,
           2,    8,    7,   19,    2,    8,    4,   13,    3,    9,   49,
           2,    6,    3,   14,    2,    1,    1,    1,    2,    1,    1,
           1,    2,    1,    1,    1,    2,   10,    3,    9,   59,   59,
          59,  268,   59,    2,   19,    4,   11,   26,    2,   13,    4,
           8,    2,    8,    3,    2,   16,   12,    3,   19,   17,    2,
           3,    2,    4,    9,    6,    4,    2,    9,    6,   12,    4,
          13,    3,    2,   16,    5,    5,   27,    4,   13,    4,    2,
         152,    1,    2,    1,    1,    1,    1,    2,  274,   65,    4,
          20,   24,    5,  217,   36,  264,   80,  120,   34,   36,   87,
          25,   47,   35,   29,   53,  153,  219,   29,   31,   62,   82,
          75,  451,   52,   23,   

In [47]:
history = model.fit(**{
    "x": X_train,
    "y": keras.utils.to_categorical(Y_train),
    "epochs": 10,
    "batch_size": 64,
    "validation_split": 0.10,
    "verbose": 1,
    })

plot_history(history)

Epoch 1/10
716/716 [==============================] - 5s 6ms/step - loss: 1.7430 - accuracy: 0.2744 - val_loss: 1.6214 - val_accuracy: 0.2840
Epoch 2/10
716/716 [==============================] - 4s 6ms/step - loss: 1.5805 - accuracy: 0.2798 - val_loss: 1.5916 - val_accuracy: 0.2840
Epoch 3/10
716/716 [==============================] - 4s 6ms/step - loss: 1.5756 - accuracy: 0.2879 - val_loss: 1.5894 - val_accuracy: 0.2840
Epoch 4/10
716/716 [==============================] - 4s 6ms/step - loss: 1.5748 - accuracy: 0.2866 - val_loss: 1.5893 - val_accuracy: 0.2840
Epoch 5/10
715/716 [============================>.] - ETA: 0s - loss: 1.5746 - accuracy: 0.2851

KeyboardInterrupt: ignored

In [ ]:
train_score = model.evaluate(X_train, tf.keras.utils.to_categorical(Y_train))
print("\n\ntrain loss: {} | train acc: {}\n".format(train_score[0], train_score[1]))

test_score = model.evaluate(X_test, tf.keras.utils.to_categorical(Y_test))
print("\n\ntest loss: {} | test acc: {}".format(test_score[0], test_score[1]))